In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mode
%matplotlib inline  

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
fr1 = pd.read_csv('csv/face_rating_1_expanded_cleaned.csv')
fr1 = fr1.drop(columns='Unnamed: 0')

fr2 = pd.read_csv('csv/face_rating_2_expanded_cleaned.csv')
fr2 = fr2.drop(columns='Unnamed: 0')

In [152]:
fr1.columns

Index(['completion_code', 'experimentCode', 'faceID', 'state', 'trialNumber',
       'uniqueID', 'participant_country', 'participant_gender',
       'participant_age', 'face_perceived_description',
       'face_perceived_typical', 'face_perceived_attractive',
       'face_perceived_photo-gender', 'face_perceived_race',
       'face_perceived_age'],
      dtype='object')

In [5]:
fr2.columns

Index(['completion_code', 'experimentCode', 'faceID', 'state', 'trialNumber',
       'uniqueID', 'participant_country', 'participant_gender',
       'participant_ethnicity-details', 'participant_ethnicity',
       'participant_age', 'face_perceived_non-physical-description',
       'face_perceived_physical-description', 'face_perceived_occupation',
       'face_perceived_typical', 'face_perceived_attractive',
       'face_perceived_photo-gender', 'face_perceived_ethnicity-details',
       'face_perceived_ethnicity', 'face_perceived_eye', 'face_perceived_hair',
       'face_perceived_age'],
      dtype='object')

In [4]:
#only use face perception columns
cols = ['faceID', 'face_perceived_physical-description',
       'face_perceived_ethnicity', 'face_perceived_ethnicity-details',
       'face_perceived_eye', 'face_perceived_hair',
       'face_perceived_photo-gender', 
       'uniqueID']

#create empty dataframe
image_fr = pd.DataFrame(columns = cols)

#add faceIDs
image_fr['faceID'] = list(fr2.faceID.unique())


for face in fr2.faceID.unique():
    for col in cols[1:]:
        col_list = fr2.loc[fr2.faceID == face, col].tolist()
        #dont add missing values
        col_list_no_nan = [x for x in col_list if str(x) != 'nan']
        image_fr.loc[image_fr['faceID'] == face, col] = [col_list_no_nan]
        

In [31]:
image_fr.head()

,faceID,face_perceived_physical-description,face_perceived_ethnicity,face_perceived_ethnicity-details,face_perceived_eye,face_perceived_hair,face_perceived_photo-gender,uniqueID,glasses_mentioned,glasses,face_gender_mode
0,IMG_8622,[Has beautiful eyebrows and a nice hair cut. H...,"[african, african, latino, latino, african, af...","[indian, Arab, Indian/Turkish, Pakistani?, Ind...","[brown, Brown, black, black, brown, Brown, bro...","[black, Black, black, black, black, Black, bla...","[male, male, male, male, male, male, male, mal...","[85762, 71137, 3200, 998, 68096, 17150, 72782,...",0,False,male
1,IMG_7942,[They have short[comma] dark hair and light br...,"[white, latino, african, latino, african, lati...","[Lebanese, Indian, Middle Eastern, Middle east...","[brown, brown, black, black, Brown, brown, bla...","[black, brown, black, black, Black, black, bla...","[male, male, male, male, male, male, male, mal...","[91607, 86889, 54955, 69543, 85473, 2015, 6647...",7,True,male
2,IMG_0401,[This is an Asian guy in his 20s. He has short...,"[african, african, african, african, african, ...",[],"[black, Brown, brown, brown, brown, black]","[black, Black, black, black, black, black]","[male, male, male, male, male, male]","[60735, 79320, 48785, 83315, 82981, 66029]",0,False,male
3,IMG_0018,[This person has a crew cut with dirty blonde ...,"[white, white, white, white, white, white, whi...",[],"[Blue, Green, blue, brown, blue, blue, green, ...","[Blonde, Blonde, blonde, blonde, dirty blonde,...","[male, male, male, male, male, male, male, mal...","[77802, 25131, 32678, 94238, 72285, 47710, 731...",0,False,male
4,IMG_0777,[long hair combed over all emo like. a beard t...,"[white, white, white, white, white, white, whi...",[],"[brown, Brown, green, black, green, black, bro...","[brown, Brown, Brown, black, brown, brown, bro...","[male, male, male, male, male, male, male, mal...","[85056, 33585, 47691, 998, 84607, 91079, 37044...",0,False,male


In [21]:
#gender
image_fr['face_gender_mode'] = image_fr['face_perceived_photo-gender'].map(mode)

In [22]:
thesis_gender_labels = image_fr[['faceID', 'face_gender_mode']].sort_values('faceID').reset_index().drop(columns='index')

In [23]:
thesis_gender_labels.head(3)

,faceID,face_gender_mode
0,IMG_0003,female
1,IMG_0018,male
2,IMG_0033,male


In [24]:
thesis_gender_labels['male'] = np.where(thesis_gender_labels['face_gender_mode']=='male', 1, 0)

In [25]:
thesis_gender_labels.head(3)

,faceID,face_gender_mode,male
0,IMG_0003,female,0
1,IMG_0018,male,1
2,IMG_0033,male,1


In [5]:
thesis_embeddings = pd.read_csv('thesis_face_embeddings/guess_who_embeddings_pandas.csv')

In [22]:
thesis_embeddings.head(2)

,encodings,faceID
0,[-0.05808836 0.03249508 0.14888 -0.041770...,IMG_0003.JPG
1,[-0.13564467 0.12020294 0.005176 -0.056101...,IMG_0018.JPG


In [6]:
emb = thesis_embeddings.encodings

emb = (emb
 .str.replace('\n','')
 .str.replace('\r','')
 .str.replace('[','')
 .str.replace(']','')
 .str.split())

emb_lists = emb.tolist()

celeb_emb_npy = np.asarray(emb_lists)

celeb_emb_npy = celeb_emb_npy.astype(np.float)

In [26]:
thesis_gender_labels.male.values

array([0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])

In [27]:
X_test = celeb_emb_npy
y_test = thesis_gender_labels.male.values

In [79]:
loaded_model = pickle.load(open('celeba/gender_model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0.9585492227979274


In [82]:
y_pred = loaded_model.predict(X_test)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.959
             precision    recall  f1-score   support

          0       1.00      0.91      0.96        94
          1       0.93      1.00      0.96        99

avg / total       0.96      0.96      0.96       193

[[86  8]
 [ 0 99]]


In [84]:
thesis_gender_labels['pred'] = y_pred

In [87]:
thesis_gender_labels.head()

,faceID,face_gender_mode,male,pred
0,IMG_0003,female,0,0
1,IMG_0018,male,1,1
2,IMG_0033,male,1,1
3,IMG_0048,female,0,0
4,IMG_0063,male,1,1


In [98]:
thesis_gender_labels['correct'] = thesis_gender_labels.male == thesis_gender_labels.pred

In [100]:
thesis_gender_labels.head(3)

,faceID,face_gender_mode,male,pred,correct
0,IMG_0003,female,0,0,True
1,IMG_0018,male,1,1,True
2,IMG_0033,male,1,1,True


In [101]:
thesis_gender_labels[thesis_gender_labels.correct == False]

,faceID,face_gender_mode,male,pred,correct
17,IMG_0295,female,0,1,False
75,IMG_7866,female,0,1,False
116,IMG_8697,female,0,1,False
125,IMG_8934,female,0,1,False
165,IMG_9574,female,0,1,False
166,IMG_9589,female,0,1,False
191,IMG_9977,female,0,1,False
192,IMG_9987,female,0,1,False


In [29]:
gender_keras_model = loaded_model = pickle.load(open('celeba/gender_keras_model.sav', 'rb'))

In [30]:
y_pred = gender_keras_model.predict(X_test)
y_pred = (y_pred > 0.5)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.959
             precision    recall  f1-score   support

          0       1.00      0.91      0.96        94
          1       0.93      1.00      0.96        99

avg / total       0.96      0.96      0.96       193

[[86  8]
 [ 0 99]]


# glass

In [112]:
image_fr['face_perceived_physical-description'].head()

0    [Hispanic female in her early to mid thirties ...
1    [Chiseled jawline [comma] neatly groomed beard...
2    [They have a high forehead a short trimmed bea...
3    [handsome and brown hair and brown eyes and ta...
4    [Pretty looking.soft nature. i think she is a ...
Name: face_perceived_physical-description, dtype: object

In [117]:
image_fr['face_perceived_physical-description'].iloc[0]

['Hispanic female in her early to mid thirties with long brown hair past the shoulders and brown eyes. ',
 'Attractive hispanic female with brown eyes and brown hair. Has nice skin and hair. She is physically fit',
 'She has long brown hair and dramatic eye brows[comma] smooth light brown skin with full lips. Skinny and tall.',
 'She is medium skin color[comma] average weight[comma] large breasted[comma] and below average height.',
 'Looks so fit in her physical appearance. her black hair combed as loosen free hair. Her lips looking as almost smiling. ',
 'She has shoulder length dark hard and is a white woman.',
 'looks to be 62-67 inches tall. Slim body type. Pretty face. ',
 "5'4''[comma] thin[comma] long hair and beautiful. She has a particular eye piercing gaze that's both wonderful and frightening."]

In [7]:
def glass_count(lst):
    count = 0
    for s in lst:
        if 'glass' in s:
            count += 1
    return count

In [8]:
image_fr['glasses_mentioned'] = image_fr['face_perceived_physical-description'].map(glass_count)

In [9]:
image_fr['glasses'] = image_fr.glasses_mentioned > 1

In [14]:
image_fr.head(2)

,faceID,face_perceived_physical-description,face_perceived_ethnicity,face_perceived_ethnicity-details,face_perceived_eye,face_perceived_hair,face_perceived_photo-gender,uniqueID,glasses_mentioned,glasses
0,IMG_8622,[Has beautiful eyebrows and a nice hair cut. H...,"[african, african, latino, latino, african, af...","[indian, Arab, Indian/Turkish, Pakistani?, Ind...","[brown, Brown, black, black, brown, Brown, bro...","[black, Black, black, black, black, Black, bla...","[male, male, male, male, male, male, male, mal...","[85762, 71137, 3200, 998, 68096, 17150, 72782,...",0,False
1,IMG_7942,[They have short[comma] dark hair and light br...,"[white, latino, african, latino, african, lati...","[Lebanese, Indian, Middle Eastern, Middle east...","[brown, brown, black, black, Brown, brown, bla...","[black, brown, black, black, Black, black, bla...","[male, male, male, male, male, male, male, mal...","[91607, 86889, 54955, 69543, 85473, 2015, 6647...",7,True


In [10]:
thesis_glas_labels = image_fr[['faceID', 'glasses']].sort_values('faceID').reset_index().drop(columns='index')

In [11]:
thesis_glas_labels.glasses.value_counts()

False    169
True      24
Name: glasses, dtype: int64

In [12]:
thesis_glas_labels['y_test'] = np.where(thesis_glas_labels['glasses']==True, 1, 0)

In [18]:
thesis_glas_labels.head(2)

,faceID,glasses,y_test
0,IMG_0003,False,0
1,IMG_0018,False,0


In [13]:
X_test = celeb_emb_npy
y_test = thesis_glas_labels['y_test'].values

In [166]:
glas_model = pickle.load(open('celeba/glasses_model.sav', 'rb'))

C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [167]:
y_pred = glas_model.predict(X_test)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.953
             precision    recall  f1-score   support

          0       0.95      0.99      0.97       169
          1       0.94      0.67      0.78        24

avg / total       0.95      0.95      0.95       193

[[168   1]
 [  8  16]]


In [168]:
thesis_glas_labels['y_pred'] = y_pred

In [185]:
thesis_glas_labels['result'] = thesis_glas_labels.y_test.astype(str) + thesis_glas_labels.y_pred.astype(str)

In [190]:
thesis_glas_labels.head()

,faceID,glasses,y_test,y_pred,result,prediction
0,IMG_0003,False,0,0,00,TN
1,IMG_0018,False,0,0,00,TN
2,IMG_0033,False,0,0,00,TN
3,IMG_0048,False,0,0,00,TN
4,IMG_0063,False,0,0,00,TN


In [189]:
def pred(result):
    if result == '00':
        return 'TN'
    if result == '11':
        return 'TP'
    
    if result == '10':
        return 'FN'
    if result == '01':
        return 'FP'
    
    


thesis_glas_labels['prediction'] = thesis_glas_labels.result.map(pred)

In [193]:
thesis_glas_labels[thesis_glas_labels.prediction == 'FN']

,faceID,glasses,y_test,y_pred,result,prediction
51,IMG_7502,True,1,0,10,FN
74,IMG_7851,True,1,0,10,FN
88,IMG_8078,True,1,0,10,FN
90,IMG_8108,True,1,0,10,FN
108,IMG_8577,True,1,0,10,FN
127,IMG_8964,True,1,0,10,FN
151,IMG_9364,True,1,0,10,FN
192,IMG_9987,True,1,0,10,FN


In [25]:
#mlp

glas_mlp_model = pickle.load(open('celeba/glasses_mlp_model.sav', 'rb'))

C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [26]:
y_pred = glas_mlp_model.predict(X_test)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.979
             precision    recall  f1-score   support

          0       1.00      0.98      0.99       169
          1       0.86      1.00      0.92        24

avg / total       0.98      0.98      0.98       193

[[165   4]
 [  0  24]]


In [15]:
glas_keras_model = pickle.load(open('celeba/glasses_keras_model.sav', 'rb'))

In [18]:
y_pred = glas_keras_model.predict(X_test)
y_pred = (y_pred > 0.5)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.979
             precision    recall  f1-score   support

          0       1.00      0.98      0.99       169
          1       0.86      1.00      0.92        24

avg / total       0.98      0.98      0.98       193

[[165   4]
 [  0  24]]


### undersample

In [158]:
glas_undersample_model = pickle.load(open('celeba/glasses_undersample_model.sav', 'rb'))

C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [160]:
y_pred = glas_undersample_model.predict(X_test)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.731
             precision    recall  f1-score   support

          0       1.00      0.69      0.82       169
          1       0.32      1.00      0.48        24

avg / total       0.91      0.73      0.78       193

[[117  52]
 [  0  24]]


In [27]:
glas_mlp_undersample_model = pickle.load(open('celeba/glasses_mlp_undersample_model.sav', 'rb'))

C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [29]:
y_pred = glas_mlp_undersample_model.predict(X_test)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.927
             precision    recall  f1-score   support

          0       0.99      0.92      0.96       169
          1       0.64      0.96      0.77        24

avg / total       0.95      0.93      0.93       193

[[156  13]
 [  1  23]]


## oversample

In [163]:
glas_oversample_model = pickle.load(open('celeba/glasses_oversample_model.sav', 'rb'))

C:\Users\Paradigit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [164]:
y_pred = glas_oversample_model.predict(X_test)
print("{:.3}".format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

0.751
             precision    recall  f1-score   support

          0       1.00      0.72      0.83       169
          1       0.33      1.00      0.50        24

avg / total       0.92      0.75      0.79       193

[[121  48]
 [  0  24]]


# eye

In [32]:
#ethnicity

#perceived  consensus
def concensus_prop(lst):
    try:
        mod = mode(lst)
        most = lst.count(mod) 
        prop = most / len(lst)
    except:
        prop = 0.5
        most = 'tie'
    return prop

#most concensus val
def concensus_val(lst):
    try:
        val = mode(lst)
        most = lst.count(val) 
        prop = most / len(lst)
    except:
        prop = 0.5
        val = 'tie' 
    return val




In [33]:
image_fr.head()

,faceID,face_perceived_physical-description,face_perceived_ethnicity,face_perceived_ethnicity-details,face_perceived_eye,face_perceived_hair,face_perceived_photo-gender,uniqueID,glasses_mentioned,glasses,face_gender_mode
0,IMG_8622,[Has beautiful eyebrows and a nice hair cut. H...,"[african, african, latino, latino, african, af...","[indian, Arab, Indian/Turkish, Pakistani?, Ind...","[brown, Brown, black, black, brown, Brown, bro...","[black, Black, black, black, black, Black, bla...","[male, male, male, male, male, male, male, mal...","[85762, 71137, 3200, 998, 68096, 17150, 72782,...",0,False,male
1,IMG_7942,[They have short[comma] dark hair and light br...,"[white, latino, african, latino, african, lati...","[Lebanese, Indian, Middle Eastern, Middle east...","[brown, brown, black, black, Brown, brown, bla...","[black, brown, black, black, Black, black, bla...","[male, male, male, male, male, male, male, mal...","[91607, 86889, 54955, 69543, 85473, 2015, 6647...",7,True,male
2,IMG_0401,[This is an Asian guy in his 20s. He has short...,"[african, african, african, african, african, ...",[],"[black, Brown, brown, brown, brown, black]","[black, Black, black, black, black, black]","[male, male, male, male, male, male]","[60735, 79320, 48785, 83315, 82981, 66029]",0,False,male
3,IMG_0018,[This person has a crew cut with dirty blonde ...,"[white, white, white, white, white, white, whi...",[],"[Blue, Green, blue, brown, blue, blue, green, ...","[Blonde, Blonde, blonde, blonde, dirty blonde,...","[male, male, male, male, male, male, male, mal...","[77802, 25131, 32678, 94238, 72285, 47710, 731...",0,False,male
4,IMG_0777,[long hair combed over all emo like. a beard t...,"[white, white, white, white, white, white, whi...",[],"[brown, Brown, green, black, green, black, bro...","[brown, Brown, Brown, black, brown, brown, bro...","[male, male, male, male, male, male, male, mal...","[85056, 33585, 47691, 998, 84607, 91079, 37044...",0,False,male


In [34]:
#eye
image_fr['eye_consensus_prop'] = image_fr.face_perceived_eye.map(concensus_prop)
image_fr['eye_perceived'] = image_fr.face_perceived_eye.map(concensus_val)

#nr of eyecolor perceived
image_fr['nr_eye_col'] = image_fr.face_perceived_eye.map(set)
image_fr['nr_eye_col'] = image_fr['nr_eye_col'].map(len)

In [37]:
image_fr.eye_perceived.value_counts()

brown    153
tie       18
Brown     12
blue       9
green      1
Name: eye_perceived, dtype: int64

In [38]:
image_fr[image_fr.eye_perceived == 'tie']

,faceID,face_perceived_physical-description,face_perceived_ethnicity,face_perceived_ethnicity-details,face_perceived_eye,face_perceived_hair,face_perceived_photo-gender,uniqueID,glasses_mentioned,glasses,face_gender_mode,eye_consensus_prop,eye_perceived,nr_eye_col
10,IMG_9921,[Short[comma] average build[comma] mid-length ...,"[african, african, african, african, african, ...",[Chinese],"[black, black, Brown, black, black, brown, bro...","[black, white, Black, black, black, black, bla...","[female, male, female, female, female, female,...","[35309, 80864, 18392, 10558, 55798, 57798, 594...",0,False,female,0.5,tie,5
11,IMG_9725,[This person is black with short black hair. ...,"[african, african, african, african, african, ...",[],"[Black, Black, brown, black, Brown, brown, Bro...","[Black, Black, black, black, Black, black, Bla...","[male, male, male, male, male, male, male, mal...","[56196, 33585, 517, 19575, 91573, 91607, 35977...",0,False,male,0.5,tie,4
19,IMG_0214,[This is an asian male with dark hair and fair...,"[african, african, african, white, african, af...",[],"[Black, Black, dark brown, brown, black, Brown...","[Brown, Black, Black, good, black, black, Blac...","[male, male, male, male, male, male, male, mal...","[23661, 30647, 7227, 81914, 44695, 59053, 2513...",0,False,male,0.5,tie,5
20,IMG_8018,[This man is tall and well built. He has curli...,"[white, white, white, white, white, white, whi...","[This person has European/Scandinavian roots, ...","[brown, blue, blue, brown, brown, Blue, Brown,...","[blonde, blonde, blond, brown, golden blonde, ...","[male, male, male, male, male, male, male, mal...","[25561, 5877, 10558, 51909, 55134, 24791, 3797...",0,False,male,0.5,tie,6
42,IMG_7745,[Young white male with red hair and green eyes...,"[white, white, white, white, latino, white, wh...","[Irish, Irish, CHRISTIAN, Irish/Scottish]","[green, br, Black, green, blue, brown, green, ...","[red, br, Brown, red, brown, brown, orange, br...","[male, male, male, male, male, male, male, mal...","[49620, 3828, 30233, 13021, 38480, 54959, 7637...",0,False,male,0.5,tie,8
71,IMG_9379,[Wide nose[comma] short hair[comma] strong eye...,"[white, latino, african, african, african, afr...","[Iranian, Indian, Arab, unsure, Indian]","[Brown, brown, Brown, Brown, brown, Brown, bro...","[Black, brown, Black, Black, black, Black, bla...","[female, female, female, female, female, femal...","[10808, 84482, 62702, 52824, 68232, 85193, 989...",0,False,female,0.5,tie,4
79,IMG_0003,[Thin diamond face[comma] thin nose[comma] rud...,"[white, white, white, white, white, white, whi...",[],"[green, Brown, brown, Grey, Brown, gray, brown...","[blonde, Dirty Blonde, brown, Blond, Dirty Blo...","[female, female, female, female, female, femal...","[8833, 20801, 21974, 11939, 18963, 59053, 7350...",0,False,female,0.5,tie,6
97,IMG_9649,[SHORT AND STOCK[comma] I WOULD SAY AROUND 190...,"[african, african, white, african, white, lati...","[INDIAN, Indian?, Middle Eastern]","[BLACK, BROWN, black, black, brown, brown, bla...","[BLACK, BLACK, black, black, black, black, bla...","[male, male, male, male, male, male, male, male]","[73885, 63872, 61594, 11505, 82621, 98149, 684...",4,True,male,0.5,tie,4
102,IMG_0716,[Round face[comma] long hair[comma] and a slig...,"[african, african, african, african, african, ...","[South korean, chinese]","[black, Brown, Brown, Black, black, brown, Bla...","[black, Black, Black, Black, black, brown, Bla...","[female, female, female, female, female, femal...","[33988, 83285, 82093, 68143, 94053, 15451, 552...",0,False,female,0.5,tie,4
113,IMG_0551,[He is a short/medium height white male. He ha...,"[white, white, white, white, white, white, whi...",[],"[Blue, green, Brown, Blue, blue, Blue, Brown, ...","[Brown, brown, Brown, Brown, brown, Brown, Bro...","[male, male, male, male, male, male, male, mal...","[17678, 56210, 40164, 54347, 48915, 20993, 303...",0,False,male,0.5,tie,5
